In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
c_drone_path = '../../data/1m/*.wav'
cm_drone_path = '../../data/10m/*.wav'
m_drone_path = '../../data/20m/*.wav'
f_drone_path = '../../data/50m/*.wav'
background_path = '../../data/background/*.wav'

c_drone_files = glob.glob(c_drone_path)
cm_drone_files = glob.glob(cm_drone_path)
m_drone_files = glob.glob(m_drone_path)
f_drone_files = glob.glob(f_drone_path)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [4]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
c_drone_raw = load(c_drone_files)
cm_drone_raw = load(cm_drone_files)
m_drone_raw = load(m_drone_files)
f_drone_raw = load(f_drone_files)
background_raw = load(background_files)

(2600960,)
(2748416,)
(2637824,)
(2723840,)
(6032517,)


# Data preprocessing

Chose `mfcc4` among functions below:
- input size 16x16; `n_mfcc=16`, used first 16 data points from 16x17 mfcc of a chunk with size 8192
- chunk siez 8192, overlapped half of it

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=22050, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
c_mfcc_drone, c_y_drone = mfcc4(c_drone_raw, 4)
cm_mfcc_drone, cm_y_drone = mfcc4(cm_drone_raw, 3)
m_mfcc_drone, m_y_drone = mfcc4(m_drone_raw, 2)
f_mfcc_drone, f_y_drone = mfcc4(f_drone_raw, 1)
mfcc_background, y_background = mfcc4(background_raw, 0)

print(c_mfcc_drone.shape, c_y_drone.shape)
print(cm_mfcc_drone.shape, cm_y_drone.shape)
print(m_mfcc_drone.shape, m_y_drone.shape)
print(f_mfcc_drone.shape, f_y_drone.shape)
print(mfcc_background.shape, y_background.shape)

(2600960,)
small end: (16, 9)
(2748416,)
small end: (16, 9)
(2637824,)
small end: (16, 9)
(2723840,)
small end: (16, 9)
(6032517,)
small end: (16, 15)
small end: (16, 7)
(634, 16, 16) (634,)
(670, 16, 16) (670,)
(643, 16, 16) (643,)
(664, 16, 16) (664,)
(1471, 16, 16) (1471,)


In [8]:
X = np.concatenate((c_mfcc_drone,cm_mfcc_drone,m_mfcc_drone,f_mfcc_drone, mfcc_background), axis=0)
y = np.hstack((c_y_drone, cm_y_drone, m_y_drone, f_y_drone, y_background))
print(X.shape, y.shape)

(4082, 16, 16) (4082,)


In [9]:
X = np.reshape(X,(X.shape[0],-1))

In [10]:
X.shape

(4082, 256)

In [11]:
n_labels = y.shape[0]
n_unique_labels = 5
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(4082, 5)


In [12]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
print(X_train.shape,y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(2612, 256) (2612, 5)
(653, 256) (653, 5)
(817, 256) (817, 5)


In [14]:
# Save Data
np.save('../../model/X_train', X_train)
np.save('../../model/X_test', X_test)
np.save('../../model/X_val', X_val)
np.save('../../model/y_val', y_val)
np.save('../../model/y_train', y_train)
np.save('../../model/y_test', y_test)

### Until this part

In [15]:
# Load Data
X_train = np.load('../../model/X_train.npy')
X_test = np.load('../../model/X_test.npy')
X_val = np.load('../../model/X_val.npy')
y_val = np.load('../../model/y_val.npy')
y_train = np.load('../../model/y_train.npy')
y_test = np.load('../../model/y_test.npy')

# Experiment 3 - One convolutional layer /w no dropout


##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [16]:
tf.reset_default_graph()

In [17]:
n_mfcc = 16
n_frame = 16
n_classes = 5
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.0002  # 0.005
training_epochs = 200 # 수정해야해

# Layer

## Experiment
- learning rate 0.0002
- no cnn/ pooling 
- just dense with dropout 

In [18]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

keep_prob = tf.placeholder(tf.float32)

dense1 = tf.layers.dense(inputs=X, units=256, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)
dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense2, keep_prob=keep_prob)
dense3 = tf.layers.dense(inputs=dropout2, units=512, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)
dense4 = tf.layers.dense(inputs=dropout3, units=512, activation=tf.nn.relu)
dropout4 = tf.nn.dropout(dense4, keep_prob=keep_prob)
dense5 = tf.layers.dense(inputs=dropout4, units=256, activation=tf.nn.relu)
dropout5 = tf.nn.dropout(dense5, keep_prob=keep_prob)

logits= tf.layers.dense(inputs=dropout5, units=5)

In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
# model save
model_path = '../../model/Dense/4_dense_model'
saver = tf.train.Saver()

# Trainning

In [22]:
import math

In [23]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
#original 
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)
with tf.device("/gpu:0"):
    for epoch in range(1,training_epochs):
        avg_cost = 0    
#    total_batch = int(X_train.shape[0] / batch_size)
        for i in range( int(math.ceil(len(X_train)/batch_size)) ): # 배치 사이즈로 나눈 나머지 다 돌 수 있게 
            x_ = X_train[batch_size * i: batch_size * (i + 1)]
            y_ = y_train[batch_size * i: batch_size * (i + 1)]
            feed_dict={X:x_, Y:y_, keep_prob:0.6}    
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            cost_history = np.append(cost_history,cost)
            avg_cost += c
            #accuracy_val = sess.run([accuracy], feed_dict={X:X_val, Y:y_val
        val_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_val , keep_prob:1})  
        val_true = sess.run(tf.argmax(y_val,1))
        accuracy_val = accuracy_score(val_pred, val_true)
        print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)), "\t검증 세트 정확도: {:.3f}%".format(accuracy_val * 100),)
saver.save(sess, model_path)

Epoch: 0001 cost =  1.232716564 	검증 세트 정확도: 43.185%
Epoch: 0002 cost =  0.397567459 	검증 세트 정확도: 39.663%
Epoch: 0003 cost =  0.203165970 	검증 세트 정확도: 45.636%
Epoch: 0004 cost =  0.130873869 	검증 세트 정확도: 49.311%
Epoch: 0005 cost =  0.091640627 	검증 세트 정확도: 47.167%
Epoch: 0006 cost =  0.069793503 	검증 세트 정확도: 57.121%
Epoch: 0007 cost =  0.058325699 	검증 세트 정확도: 62.787%
Epoch: 0008 cost =  0.051268557 	검증 세트 정확도: 67.381%
Epoch: 0009 cost =  0.046255898 	검증 세트 정확도: 70.291%
Epoch: 0010 cost =  0.043543846 	검증 세트 정확도: 75.651%
Epoch: 0011 cost =  0.038959113 	검증 세트 정확도: 75.498%
Epoch: 0012 cost =  0.037450718 	검증 세트 정확도: 79.173%
Epoch: 0013 cost =  0.035204150 	검증 세트 정확도: 75.345%
Epoch: 0014 cost =  0.033616620 	검증 세트 정확도: 84.074%
Epoch: 0015 cost =  0.032714984 	검증 세트 정확도: 77.489%
Epoch: 0016 cost =  0.031485110 	검증 세트 정확도: 77.795%
Epoch: 0017 cost =  0.029792272 	검증 세트 정확도: 82.542%
Epoch: 0018 cost =  0.028756371 	검증 세트 정확도: 84.533%
Epoch: 0019 cost =  0.027529770 	검증 세트 정확도: 80.858%
Epoch: 0020 

Epoch: 0159 cost =  0.002194703 	검증 세트 정확도: 96.478%
Epoch: 0160 cost =  0.002165653 	검증 세트 정확도: 97.397%
Epoch: 0161 cost =  0.001860174 	검증 세트 정확도: 97.703%
Epoch: 0162 cost =  0.002125740 	검증 세트 정확도: 97.090%
Epoch: 0163 cost =  0.002368746 	검증 세트 정확도: 97.550%
Epoch: 0164 cost =  0.002585642 	검증 세트 정확도: 97.243%
Epoch: 0165 cost =  0.002148813 	검증 세트 정확도: 97.397%
Epoch: 0166 cost =  0.001503790 	검증 세트 정확도: 96.937%
Epoch: 0167 cost =  0.002047661 	검증 세트 정확도: 97.550%
Epoch: 0168 cost =  0.001464659 	검증 세트 정확도: 97.703%
Epoch: 0169 cost =  0.001779837 	검증 세트 정확도: 97.397%
Epoch: 0170 cost =  0.001768472 	검증 세트 정확도: 97.703%
Epoch: 0171 cost =  0.001738973 	검증 세트 정확도: 97.397%
Epoch: 0172 cost =  0.001260148 	검증 세트 정확도: 97.703%
Epoch: 0173 cost =  0.001650039 	검증 세트 정확도: 97.550%
Epoch: 0174 cost =  0.001933097 	검증 세트 정확도: 96.018%
Epoch: 0175 cost =  0.002132104 	검증 세트 정확도: 97.397%
Epoch: 0176 cost =  0.001716921 	검증 세트 정확도: 97.090%
Epoch: 0177 cost =  0.001919514 	검증 세트 정확도: 97.550%
Epoch: 0178 

'../../model/Dense/4_dense_model'

## Prediction

In [26]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test , keep_prob:1}) 
y_true = sess.run(tf.argmax(y_test,1))

In [27]:
# Ptint Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.968
Accuracy:  0.9681762545899633
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       275
           1       0.95      0.97      0.96       132
           2       0.92      0.98      0.95       135
           3       0.97      0.85      0.91       131
           4       1.00      1.00      1.00       144

   micro avg       0.97      0.97      0.97       817
   macro avg       0.96      0.96      0.96       817
weighted avg       0.97      0.97      0.97       817

[[275   0   0   0   0]
 [  1 128   0   3   0]
 [  2   0 132   1   0]
 [  1   7  11 112   0]
 [  0   0   0   0 144]]
